In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import seaborn as sns
import plotly.express as px
import requests

In [ ]:
df = pd.read_parquet('dataset_clean.parquet')

pd.set_option('display.max_columns', None)

df.head()

In [ ]:
df.info()

---

#### Tri des valeurs des effectifs

In [ ]:
df["Tranche de l'effectif de l'unité légale"].unique()

In [ ]:
df["Tranche de l'effectif de l'unité légale"] = df["Tranche de l'effectif de l'unité légale"].str.strip()

# Mapping des tranches d'effectif à des valeurs numériques
mapping = {
    "0 salarié": 0,
    "Etablissement non employeur": 0,
    "1 ou 2 salariés": 1,
    "3 à 5 salariés": 3,
    "6 à 9 salariés": 6,
    "10 à 19 salariés": 10,
    "20 à 49 salariés": 20,
    "50 à 99 salariés": 50,
    "100 à 199 salariés": 100,
    "250 à 499 salariés": 250
}

df['Tranche_effectif_num'] = df["Tranche de l'effectif de l'unité légale"].map(mapping).astype('Int64')

# 4. Vérification : quelles valeurs n'ont pas été mappées ?
valeurs_non_mappees = df[df['Tranche_effectif_num'].isna()]["Tranche de l'effectif de l'unité légale"].unique()
if len(valeurs_non_mappees) > 0:
    print("⚠️ Certaines valeurs n'ont pas été mappées :", valeurs_non_mappees)
else:
    print("✅ Toutes les valeurs ont été mappées correctement.")

# 5. Vérification finale
df[["Tranche de l'effectif de l'unité légale", "Tranche_effectif_num"]].head(10)


In [ ]:
# dropper la colonne originale
df=df.drop(columns=['Tranche de l\'effectif de l\'unité légale'])

In [ ]:
df['Tranche_effectif_num'].value_counts()

---

#### Changement de format pour les dates avec vérification

In [ ]:
df.dtypes

In [ ]:
# Liste des colonnes à convertir (on exclut celles déjà en datetime si tu veux gagner du temps)
cols_datetime = [
    "Date de création de l'établissement",
    "Date de la dernière mise à jour de l'établissement",
    "Date du début de la période de l'établissement",
    "Date de création de l'unité légale",
    "Date du dernier traitement de l'unité légale",
    "Date de début de l'unité légale",
    "Date_fermeture_finale"
]

for col in cols_datetime:

    df[col] = pd.to_datetime(df[col], errors='coerce', utc=True)
    
    # 2. On retire l'information de fuseau horaire (tz_localize(None)) 

    if pd.api.types.is_datetime64_any_dtype(df[col]):
        df[col] = df[col].dt.tz_localize(None).dt.normalize()

print("✅ Conversions terminées !")

---

#### Analyse des fermetures

In [ ]:
# Préparer les données avec labels lisibles
df_counts = df["Catégorie juridique de l'unité légale"].value_counts().reset_index()
df_counts.columns = ["code_juridique", "nombre"]

# Remplacer les codes par les noms
label_map = {
    "5710": "SAS",
    "5499": "SARL"
}
df_counts["code_juridique"] = df_counts["code_juridique"].map(lambda x: label_map.get(str(x), x))

fig = px.pie(
    df_counts,
    names="code_juridique",
    values="nombre",
    width=700,
    title="Répartition des catégories juridiques fermées"
)

fig.show()

In [ ]:
# 1️⃣ Histogramme des périodes de l'unité légale
fig1 = px.histogram(
    df,
    x="Nombre de périodes de l'unité légale",
    nbins=40,
    title="Répartition des périodes de l'unité légale",
    width=700,
    labels={"Nombre de périodes de l'unité légale": "Nombre de périodes"}
)
fig1.show()

In [ ]:
# Pie chart interactif

df_counts = df["Tranche_effectif_num"].value_counts().sort_index().reset_index()
df_counts.columns = ["tranche", "nombre"]

fig2_pie = px.pie(
    df_counts,
    names="tranche",
    values="nombre",
    title="Répartition proportionnelle des tranches d'effectif",
    color="tranche",
    width=700,
    height=700
)
fig2_pie.show()

---

#### Traitement des dates pour obtenir une ancienneté la plus pertinente possible

In [ ]:
# Colonnes de démarrage
cols_dates = [
    "Date de création de l'unité légale",
    "Date de début de l'unité légale", 
    "Date de création de l'établissement"
]

# Paramètres
borne_min = pd.Timestamp("1990-01-01")
date_aberrante = pd.Timestamp("1900-01-01")
date_ref = pd.Timestamp("2025-12-31")

# --- Étape 1 : Nettoyage des dates aberrantes ---
for col in cols_dates:
    df[col] = pd.to_datetime(df[col], errors="coerce")
    df.loc[df[col] == date_aberrante, col] = pd.NaT
    df.loc[df[col] < borne_min, col] = pd.NaT

# --- Étape 2 : Détermination de la date de fin réelle ---

date_fin_effective = df["Date_fermeture_finale"].fillna(date_ref)

# --- Étape 3 : Calcul des âges en années (basé sur la fin réelle) ---

df["age_unite_legale"] = (date_fin_effective - df["Date de création de l'unité légale"]).dt.days / 365.25
df["age_debut_unite"] = (date_fin_effective - df["Date de début de l'unité légale"]).dt.days / 365.25
df["age_etablissement"] = (date_fin_effective - df["Date de création de l'établissement"]).dt.days / 365.25

# --- Étape 4 : Estimation unique de l'ancienneté ---

df["age_estime"] = df[["age_unite_legale", "age_debut_unite", "age_etablissement"]].max(axis=1)

# --- Étape 5 : Nettoyage final et conversion ---

df["age_estime"] = df["age_estime"].round(0).astype("Int64")

# Indicateur de fiabilité
df["age_estime_incertain"] = df[cols_dates].isna().all(axis=1).astype(int)

# --- Étape 6 : Suppression des colonnes intermédiaires ---

df = df.drop(columns=cols_dates + ["age_unite_legale", "age_debut_unite", "age_etablissement"])

# --- Vérification rapide ---
print(df[["age_estime", "age_estime_incertain"]].head())

In [ ]:
df['age_estime_incertain'].value_counts()

In [ ]:
df=df.drop(columns=['age_estime_incertain'])
df.head()

---

##### Sortie des effectifs de plus de 49 salariés

In [ ]:
df_filtered = df[
    (df["Tranche_effectif_num"] < 50)
     ]
df_filtered.shape

##### Visu des âges

In [ ]:
# Comptage du nombre de sociétés par âge
age_counts = df_filtered["age_estime"].value_counts().sort_index().reset_index()
age_counts.columns = ["age", "nombre_societes"]

# Barplot interactif
fig = px.bar(
    age_counts,
    x="age",
    y="nombre_societes",
    title="Répartition du nombre de sociétés fermées par âge",
    labels={"age": "Âge (années)", "nombre_societes": "Nombre de sociétés"},
    text="nombre_societes",
    height=500,
    width=1200
)

# Affichage du nombre sur les barres
fig.update_traces(textposition="outside")

# Affichage
fig.show()

##### Visu âge et effectifs

In [ ]:
# Grouper par âge et tranche d'effectif
df_counts = (
    df_filtered.groupby(['age_estime', 'Tranche_effectif_num'])
      .size()
      .reset_index(name='nombre_societes')
)

# Ordre croissant des tranches d'effectif
effectif_order = sorted(df_counts["Tranche_effectif_num"].unique())

# Scatter interactif avec échelle de couleurs adaptée
fig = px.scatter(
    df_counts,
    x='age_estime',
    y='Tranche_effectif_num',
    size='nombre_societes',
    color='nombre_societes',               
    color_continuous_scale='Viridis',
    hover_data=['nombre_societes'],
    title="Nombre de sociétés fermées par âge et tranche d'effectif",
    labels={
        'age_estime': "Âge estimé (années)",
        'Tranche_effectif_num': "Tranche d'effectif",
        'nombre_societes': "Nombre de sociétés"
    },
    size_max=40,
    height=600,
    width=1200
)

fig.update_yaxes(type='category', categoryorder='array', categoryarray=effectif_order)

fig.show()

---

#### visu de l'ESS

In [ ]:
# Compter les occurrences
df_ess_counts = df_filtered["Economie sociale et solidaire unité légale"].value_counts().reset_index()
df_ess_counts.columns = ["Economie sociale et solidaire unité légale", "count"]

fig = px.pie(
    df_ess_counts,
    values="count",
    names="Economie sociale et solidaire unité légale",
    width=700,
    title="Répartition de l'ESS après 3 exercices"
)

fig.show()


---

In [ ]:
df_filtered.dtypes

#### Visu géographique des fermetures

In [ ]:
df_map = df_filtered.copy()

# 1. On sépare en deux colonnes
coords = df_map["Géolocalisation de l'établissement"].str.split(",", expand=True)

# 2. On convertit en float de manière sécurisée
df_map["latitude"] = pd.to_numeric(coords[0], errors='coerce')
df_map["longitude"] = pd.to_numeric(coords[1], errors='coerce')

# 3. On nettoie les lignes sans coordonnées
df_map = df_map.dropna(subset=["latitude", "longitude"])

In [ ]:
# drop colonne inutile
df_map=df_map.drop(columns=['Géolocalisation de l\'établissement'])
df_map.head()

---

#### Heatmap par département

In [ ]:
df_dep = (
    df_map.groupby("Code du département de l'établissement")
      .size()
      .reset_index(name="nb_etablissements")
)
df_dep.head()


In [ ]:
# 1️⃣ Charger le GeoJSON des départements depuis le web
url = "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
r = requests.get(url)
geojson_dep = r.json()

# 2️⃣ Préparer les données : compter les établissements par département
df_dep = df.groupby("Code du département de l'établissement").size().reset_index(name="nb_etablissements")

# 3️⃣ S'assurer que les codes sont bien au format 2 caractères
df_dep["Code du département de l'établissement"] = df_dep["Code du département de l'établissement"].astype(str).str.zfill(2)

# 4️⃣ Créer la carte choroplèthe
fig = px.choropleth(
    df_dep,
    geojson=geojson_dep,
    locations="Code du département de l'établissement",
    featureidkey="properties.code",
    color="nb_etablissements",
    color_continuous_scale="Reds",
    scope="europe",
    labels={"nb_etablissements": "Nombre d'établissements"},
    title="Répartition des établissements par département",
    width=800,
)

# 5️⃣ Ajuster l'affichage
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0, "t":40, "l":0, "b":0})

fig.show()

---

In [ ]:
df_map.shape

---

#### Export en Parquet

In [ ]:
# 1. On s'assure que les colonnes de coordonnées sont propres
# et on supprime les colonnes inutiles
df_map["latitude"] = df_map["latitude"].astype(float)
df_map["longitude"] = df_map["longitude"].astype(float)

df_map.to_parquet('dataset_map.parquet', index=False)

print("✅ Fichier 'dataset_map.parquet' sauvegardé avec succès.")

---